In [1]:
!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json

--2024-05-26 18:07:54--  https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json [following]
--2024-05-26 18:07:54--  https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.1’

documents.json.1    100%[===================>] 642.90K  3.15MB/s    in 0.2s    

2024-05-26 18:07:55 (3.15 MB/s) - ‘documents.json.1’ saved [65

In [2]:
!head documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {


In [3]:
import json

with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
len(documents)

948

In [5]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': 'a582fe9cc286', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'udRh0lqyTG-qdiwmbRqEDw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/5ZwrDyW4SiaB4auA808xhg] already exists')

In [10]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 42.90it/s]


In [11]:
user_question = "How do I join the course after it has started?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [12]:
response = es.search(index=index_name, body=search_query)

In [13]:
for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section

In [14]:
es = Elasticsearch("http://localhost:9200")
def retrieve_documents(query, index_name="course-questions", max_results=5):

    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [15]:
user_question = "How do I join the course after it has started?"

response = retrieve_documents(user_question)

for doc in response:
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section

In [17]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What is the main issue with the world today?"}]
)
print(response.choices[0].message.content)

The question of what the main issue facing the world today is a complex and subjective one, as it depends on one's perspective, priorities, and values. However, several significant challenges are often highlighted as pressing concerns:

1. **Climate Change and Environmental Degradation**: The impact of human activities on the environment is causing global warming, extreme weather events, loss of biodiversity, and pollution. These issues threaten ecosystems, human health, and economies worldwide.
   
2. **Inequality**: Economic disparities and social inequalities persist both within and between countries. This includes disparities in wealth, education, healthcare access, and opportunities, leading to social tension and impeding overall development.

3. **Global Health Crises**: The COVID-19 pandemic has exposed vulnerabilities in global health systems, highlighting the need for improved healthcare infrastructure, better pandemic preparedness, and equitable access to vaccines and treatme

In [18]:
context_docs = retrieve_documents(user_question)

context = ""

for doc in context_docs:
    doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    context += doc_str

context = context.strip()
print(context)

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: G

In [20]:
prompt = f"""
You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contain the answer, return "NONE"

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

In [21]:
prompt

'You\'re a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. \nOnly use the facts from the CONTEXT. If the CONTEXT doesn\'t contain the answer, return "NONE"\n\nQUESTION: How do I join the course after it has started?\n\nCONTEXT:\n\nSection: General course-related questions\nQuestion: Course - Can I still join the course after the start date?\nAnswer: Yes, even if you don\'t register, you\'re still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.\n\nSection: General course-related questions\nQuestion: Course - Can I still join the course after the start date?\nAnswer: Yes, even if you don\'t register, you\'re still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.\n\nSection: General course

In [22]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)
answer = response.choices[0].message.content
answer

"The CONTEXT indicates that you can still join the course after it has started. Yes, even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

In [23]:
#Putting everything together

def build_context(documents):
    context = ""

    for doc in documents:
        doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
        context += doc_str
    
    context = context.strip()
    return context


def build_prompt(user_question, documents):
    context = build_context(documents)
    return f"""
You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

def ask_openai(prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer

def qa_bot(user_question):
    context_docs = retrieve_documents(user_question)
    prompt = build_prompt(user_question, context_docs)
    answer = ask_openai(prompt)
    return answer

In [24]:
qa_bot("I'm getting invalid reference format: repository name must be lowercase")

'The error "invalid reference format: repository name must be lowercase" typically occurs when you\'re working with Docker on Windows and the repository name includes uppercase letters or spaces. Mapping volumes on Windows can be particularly tricky. Here are a few solutions you could try:\n\n1. **Move Your Data Directory**: Ensure that your data is in a directory path without spaces. For example, if your code is in `C:/Users/Alexey Grigorev/git/...`, move it to something like `C:/git/...`.\n\n2. **Replace the `-v` Part**: Try different volume mapping options:\n    - `-v /c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data`\n    - `-v //c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data`\n    - `-v /c/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data`\n    - `-v //c/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data`\n    - `--volume //driveletter/path/ny_taxi_postgres_data/:/var/lib/postgresql/data`\n\n3. **Add `winpty` Before the Docker Command** (if usi

In [25]:
qa_bot("I can't connect to postgres port 5432, my password doesn't work")

"It appears the issue you're encountering may be due to a port conflict or an incorrect password for your Postgres setup. Here are several steps you can take to resolve it:\n\n1. **Change the Port**: The port 5432 might be occupied by another instance. You’re connecting to the port on your machine, not in Docker. Try substituting port 5432 with 5431 (or any other free port you have mapped).\n\n2. **Check for Running Services**: If the error persists, check if you have a Postgres service running on your machine. On Windows, you can stop this service to resolve the conflict.\n\n3. **Verify Credentials**: Make sure that the username and password you are using are correct. You can verify user access by executing the following commands inside your Docker container:\n    ```bash\n    docker exec -it <your_container_name> /bin/bash\n    psql -U root -d ny_taxi\n    ```\n\n4. **Alternative Solutions**:\n    - Change `POSTGRES_USER=root` to `PGUSER=postgres` in your Docker configuration if appl

In [26]:
qa_bot("how can I run kafka?")

"To run a Kafka producer, consumer, or streams application in the terminal using Java, follow these steps:\n\n1. **Navigate to your project directory.**\n2. **Run the following command in the terminal:**\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your JAR file. The command should be executed from within your project's directory.\n\nIf you need more specific guidance or run into any issues, please provide further details about your setup or the specific component you are trying to run."

In [27]:
qa_bot("How do I install docker locally on macos")

'The provided CONTEXT does not contain specific information on installing Docker locally on macOS. Please refer to the official Docker documentation for detailed installation steps. Generally, you can install Docker on macOS by downloading and running Docker Desktop:\n\n1. Visit the Docker Desktop website: https://www.docker.com/products/docker-desktop\n2. Click on "Download for Mac" and follow the instructions.\n\nFor a complete guide, visit the official Docker documentation: https://docs.docker.com/desktop/install/mac-install/\n\nIf you encounter issues, you might need to ensure that any virtualization features required by Docker are properly configured, similar to the nested virtualization setup mentioned for VMs in the provided CONTEXT. However, normally this shouldn\'t be necessary for standard Docker Desktop installations on macOS.'